In [3]:
from pathlib import Path
import joblib
from sentence_transformers import SentenceTransformer

MODEL_FILE = Path("svm.joblib")
LOCAL_EMBEDDER = Path("/opt/huggingface_models/all-MiniLM-L6-v2")
FALLBACK_EMBEDDER = "all-MiniLM-L6-v2"


In [6]:
clf = joblib.load(MODEL_FILE)
clf

SVC(kernel='linear')

In [7]:
embedder_id = str(LOCAL_EMBEDDER) if LOCAL_EMBEDDER.exists() else FALLBACK_EMBEDDER
embedder_id


'all-MiniLM-L6-v2'

In [8]:
model = SentenceTransformer(embedder_id)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
headlines_path = Path("headlines_nyt_2024-12-02.txt")  # change if needed
headlines = [line.strip() for line in headlines_path.read_text(encoding="utf-8").splitlines() if line.strip()]
len(headlines), headlines[:5]


(60,
 ['Biden Issues a ‘Full and Unconditional Pardon’ of His Son Hunter. After pledging not do so amid President-elect Trump’s attacks, President Biden ended Hunter Biden’s legal woes, including a guilty verdict in a gun case.',
  'Analysis. In Pardoning His Son, Biden Echoes Some of Trump’s Complaints',
  'Hunter Biden Faced Prison Time for Tax and Gun Charges. 1 min read',
  'The HeadlinesAudio. Biden Pardons His Son in U-Turn, Syrian Rebels Advance, and More',
  'Analysis. Trump Remains Defiant After the Collapse of the Matt Gaetz Selection'])

In [10]:
embeddings = model.encode(headlines, convert_to_numpy=True, show_progress_bar=False)
labels = clf.predict(embeddings)

list(zip(labels[:10], headlines[:10]))


[('Neutral',
  'Biden Issues a ‘Full and Unconditional Pardon’ of His Son Hunter. After pledging not do so amid President-elect Trump’s attacks, President Biden ended Hunter Biden’s legal woes, including a guilty verdict in a gun case.'),
 ('Neutral',
  'Analysis. In Pardoning His Son, Biden Echoes Some of Trump’s Complaints'),
 ('Neutral',
  'Hunter Biden Faced Prison Time for Tax and Gun Charges. 1 min read'),
 ('Neutral',
  'The HeadlinesAudio. Biden Pardons His Son in U-Turn, Syrian Rebels Advance, and More'),
 ('Neutral',
  'Analysis. Trump Remains Defiant After the Collapse of the Matt Gaetz Selection'),
 ('Neutral',
  'Schumer Presses for F.B.I. Checks and Senate Consideration of Trump Nominees. In a letter, Senator Chuck Schumer said Democrats would work with Republicans, but asserted that Donald Trump’s picks should undergo Senate vetting.'),
 ('Neutral',
  'Kash Patel Would Bring Bravado and Baggage to F.B.I. Role. 6 min read'),
 ('Neutral',
  'Kash Patel Would Bring Bravado 

In [11]:
import datetime as dt

source = "nyt"
today = dt.date.today()
out_path = Path(f"headline_scores_{source}_{today:%Y_%m_%d}.txt")

with out_path.open("w", encoding="utf-8") as f:
    for label, headline in zip(labels, headlines):
        f.write(f"{label},{headline}\n")

out_path


PosixPath('headline_scores_nyt_2026_01_26.txt')